In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
import time
import requests

In [119]:
def find_sector_ids(url):
    """Function to scrap sector IDs in elements with onclick property from Banxico.

    Parameters
    ----------
    url : str
        banxico url containig the series id information

    Returns
    -------
    sector_ids : list
        list containing sector ids as strings
    """
    sector_ids = []
    try:
        driver = webdriver.Chrome()
        driver.get(url)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(5)
        
        #driver.find_element_by_xpath(".//input[contains(@onclick, '1 Bedroom Deluxe')]")
        elements = driver.find_elements(By.XPATH, '//*[@onclick]')

        if len(elements) < 3: 
            driver.quit()
            time.sleep(10)
            find_sector_ids(url)


        for element_id in elements:
            onclick_txt = element_id.get_attribute('onclick')
            if "cargaDirectorioSector" in onclick_txt:
                sector_ids.append((onclick_txt.split(",")[0].split("(")[-1]))
        return sector_ids

    finally:
        driver.quit()


banxico_series_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries" 

sector_ids = find_sector_ids(banxico_series_url)



In [ ]:
sectores_api_url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/cat/sectores/"
def find_cuadroid_by_sectorid(sectors_ids):
    id_cuadros = []

    def recursive_func(item):
        if isinstance(item, dict):
            if 'idCuadro' in item:
                id_cuadros.append(item['idCuadro'])
            if 'cuadros' in item:
                recursive_func(item['cuadros'])
        elif isinstance(item, list):
            for i_t in item:
                recursive_func(i_t)
                
    for sector_id in sectors_ids:
        api_url = sectores_api_url+sector_id
        response = requests.get(api_url, headers={"Accept": "application/json"})

        if response.status_code == 200:
            data = response.json()
            recursive_func(data['bmx']['cuadros']) 
        else:
            id_cuadros.append('')
    
    return id_cuadros
        

cuadros_ids = find_cuadroid_by_sectorid(sector_ids)



In [163]:
cuadros_api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/cat/estructuras/"
def find_serieid_by_cuadrorid(cuadros_list):
    found_series = []
    def recursive_func(item):
        if isinstance(item, dict):
            if 'idSerie' in item:
                if (item['idSerie'].strip() != "") and (item['idSerie'] not in found_series):
                    found_series.append(item['idSerie'])
            for value in item.values():
                recursive_func(value)
        elif isinstance(item, list):
            for i_t in item:
                recursive_func(i_t)
    
    for c_id in cuadros_list:
        api_url = cuadros_api_url+c_id
        response = requests.get(api_url, headers={"Accept": "application/json"})

        if response.status_code == 200:
            data = response.json()
            recursive_func(data['bmx']) 
        else:
            pass
    
    return found_series
                
series_ids = find_serieid_by_cuadrorid(cuadros_ids)

series_ids

['SF43770',
 'SF43771',
 'SF43769',
 'SF44042',
 'SF44043',
 'SF44044',
 'SF43695',
 'SF43702',
 'SF43696',
 'SF43671',
 'SF43672',
 'SF43673',
 'SF43658',
 'SF43686',
 'SF43687',
 'SF63315',
 'SF43688',
 'SF43689',
 'SF43968',
 'SF43690',
 'SF43691',
 'SF43692',
 'SF43693',
 'SF43657',
 'SF267992',
 'SF364342',
 'SF44081',
 'SF65183',
 'SF44032',
 'SF43703',
 'SF43704',
 'SF43705',
 'SF43706',
 'SF43707',
 'SF43708',
 'SF43709',
 'SF43710',
 'SF43711',
 'SF6',
 'SF1',
 'SF4',
 'SF5',
 'SF1574',
 'SF7',
 'SF8',
 'SF9',
 'SF63327',
 'SF1575',
 'SF1576',
 'SF1577',
 'SF26619',
 'SF236017',
 'SF1580',
 'SF1578',
 'SF1579',
 'SF1583',
 'SF1581',
 'SF42853',
 'SF1582',
 'SF10740',
 'SF1587',
 'SF269134',
 'SF364369',
 'SF1584',
 'SF65208',
 'SF1585',
 'SF29652',
 'SF29653',
 'SF29654',
 'SF29655',
 'SF29656',
 'SF29657',
 'SF29658',
 'SF29659',
 'SF29660',
 'SF43659',
 'SF43660',
 'SF43661',
 'SF43663',
 'SF63312',
 'SF43662',
 'SF63313',
 'SF43664',
 'SF43665',
 'SF43666',
 'SF43667',
 'SF

In [164]:
len(series_ids)

128871